### Plot sample LFR graphs with ground truth and predicted communities

In this notebook we plot sample LFR graphs with nodes clustered according to their ground truth communities. We also
visualize prediction results to enable insight into the behavior of the detection methods.

First, we import the necessary packages.

In [1]:
%reload_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
from clusim.clustering import Clustering
from matplotlib.pyplot import Line2D

from src.utils.cluster_analysis import matched_memberships, get_misclassfied_nodes
from src.utils.plotting import init_plot_style, plot_graph, community_layout
from src.wrappers.igraph import read_graph
from src.wrappers.metrics import ami_score

%pylab

fig_dir = '../figures/'
init_plot_style()

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


Select and load a sample graph.

In [3]:
avg_degree = 25
n = 600
mu = 0.55
seed = 296180695#106632307#27516119#10997762#14615223


graph_file = f'../data/lfr_benchmark/{avg_degree}deg/{n}n/{int(100*mu)}mu/graph_{seed}.txt'
clu_file_true = f'../data/lfr_benchmark/{avg_degree}deg/{n}n/{int(100*mu)}mu/clustering_{seed}.json'
clu_file_synwalk = f'../results/lfr/clustering/synwalk/{avg_degree}deg/{n}n/{int(100*mu)}mu/clustering_{seed}_synwalk.json'
clu_file_walktrap = f'../results/lfr/clustering/walktrap/{avg_degree}deg/{n}n/{int(100*mu)}mu/clustering_{seed}_walktrap.json'

# load graph + true clustering
graph = read_graph(graph_file)
clu_true = Clustering().load(clu_file_true)

print(f'Synwalk AMI score is {ami_score(clu_file_synwalk, clu_file_true, graph_file)}')
print(f'Walktrap AMI score is {ami_score(clu_file_walktrap, clu_file_true, graph_file)}')

Synwalk AMI score is 0.9409424671702553
Walktrap AMI score is 0.8736630728877819


Plot the sample graph with the predicted clusterings.

In [4]:
# plot setup
plt.close('all')
save_figures = True
cmap = plt.get_cmap('Set3')

# generate custom layout
layout=community_layout(clu_true)

# plot ground truth
fig, ax = plt.subplots(figsize=(12,9))
plot_graph(graph_file, clu_true.to_membership_list(), layout, ax, cmap=cmap)
plt.tight_layout()
if save_figures:
    fig_name = 'lfr_sample_true.pdf'
    fig.savefig(fig_dir + fig_name, dpi=600, format='pdf')
    plt.close()

# extract misclassified and residual nodes for synwalk prediction
clu_pred = Clustering().load(clu_file_synwalk)
_, misclassified = get_misclassfied_nodes(clu_true, clu_pred)
residual_nodes_cluster, modified_memberlist = matched_memberships(clu_true, clu_pred)
residual_nodes = [node for node, cluster in enumerate(modified_memberlist) if cluster == residual_nodes_cluster]
residual_clusters = max(clu_pred.n_clusters - clu_true.n_clusters, 0)

# plot synwalk prediction
fig, ax = plt.subplots(figsize=(12,9))
plot_graph(graph_file, modified_memberlist, layout, ax, list(misclassified), cmap)

# assemble custom legend for community colors
custom_lines = [Line2D([], [], color=cmap(residual_nodes_cluster), ms=15, lw=0, marker='o'),
                Line2D([], [], color='black', ms=15, lw=0, marker='o', markerfacecolor='white')]
custom_labels = [f'Residual nodes ({len(residual_nodes)} nodes in {residual_clusters} clusters)',
                 f'Misclassified nodes ({len(misclassified)} nodes)']
ax.legend(custom_lines, custom_labels, loc='upper center', bbox_to_anchor=(0.5, 1.2))
plt.tight_layout()

if save_figures:
    fig_name = 'lfr_sample_synwalk.pdf'
    fig.savefig(fig_dir + fig_name, dpi=600, format='pdf')
    plt.close()

# extract misclassified and residual nodes for walktrap prediction
clu_pred = Clustering().load(clu_file_walktrap)
_, misclassified = get_misclassfied_nodes(clu_true, clu_pred)
residual_nodes_cluster, modified_memberlist = matched_memberships(clu_true, clu_pred)
residual_nodes = [node for node, cluster in enumerate(modified_memberlist) if cluster == residual_nodes_cluster]
residual_clusters = max(clu_pred.n_clusters - clu_true.n_clusters, 0)

# plot walktrap prediction
fig, ax = plt.subplots(figsize=(12,9))
plot_graph(graph_file, modified_memberlist, layout, ax, list(misclassified), cmap)

# assemble custom legend for community colors
custom_lines = [Line2D([], [], color=cmap(residual_nodes_cluster), ms=15, lw=0, marker='o'),
                Line2D([], [], color='black', ms=15, lw=0, marker='o', markerfacecolor='white')]
custom_labels = [f'Residual nodes ({len(residual_nodes)} nodes in {residual_clusters} clusters)',
                 f'Misclassified nodes ({len(misclassified)} nodes)']
ax.legend(custom_lines, custom_labels, loc='upper center', bbox_to_anchor=(0.5, 1.2))
plt.tight_layout()

if save_figures:
    fig_name = 'lfr_sample_walktrap.pdf'
    fig.savefig(fig_dir + fig_name, dpi=600, format='pdf')
    plt.close()